# Лабораторная работа №3 "Дообучение на основе GPT-2"

In [ ]:
# https://habr.com/ru/articles/859250/#comment_27576236

from transformers import (
    GPT2LMHeadModel,
    GPT2Tokenizer,
    TextDataset,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments
)
from os import path
import matplotlib.pyplot as plt
import re
import requests
import torch


FILENAME = r'data/gariki_igor_guberman.txt'
END_OF_TEXT_TOKEN = '<|endoftext|>'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def export_data() -> str:
    URL = 'https://www.booksite.ru/localtxt/gub/erm/an/guberman_i/gariki/'
    PAGE_COUNT = 3

    PRE_PATTERN = re.compile(r'<pre>(.*?)</pre>', re.S)
    TAG_PATTERN = re.compile(r'<.*?>', re.S)
    MULTI_SPACE_PATTERN = re.compile(r'\s+')

    result = []
    for page in range(1, PAGE_COUNT + 1):
        url = f'{URL}/{page}.htm'

        with requests.get(url) as response:
            response.encoding = 'cp1251'
            for pre in PRE_PATTERN.finditer(response.text):
                for line in TAG_PATTERN.sub('', pre[1]).split('\n'):
                    striped = line.strip()
                    striped = MULTI_SPACE_PATTERN.sub(' ', striped)
                    result.append(striped if striped else END_OF_TEXT_TOKEN)

    return '\n'.join(result)


def build_poem(generated_text: str) -> str:
    SENTENCE_COUNT = 4
    NEW_LINES_PATTERN = re.compile(r'\n+')
    TAG_PATTERN = re.compile(r'<.*?>', re.S)

    result = NEW_LINES_PATTERN.sub('\n', generated_text)
    result = TAG_PATTERN.sub('', result)
    result = '\n'.join(line.strip() for line in result.split('\n')[:SENTENCE_COUNT])

    return result

## Подготовка данных

In [ ]:
try:
    with open(FILENAME, 'r', encoding='utf-8') as file:
        data = file.read()
except Exception:
    data = export_data()
    with open(FILENAME, 'w', encoding='utf-8') as file:
        file.write(data)

In [ ]:
poems = [
    [word for word in re.findall(r'\b(\w*?)\b', poem.strip()) if word]
    for poem in data.split(END_OF_TEXT_TOKEN)
]
poems_len = [len(poem) for poem in poems]

plt.hist(poems_len, bins=10, color='blue', edgecolor='black')

plt.xlabel('Число слов')
plt.ylabel('Частота')
plt.title('Число слов приходящихся на одно стихотворение')

plt.show()

## Обучение модели

In [ ]:
PRETRAINED_MODEL_PATH = 'sberbank-ai/rugpt3large_based_on_gpt2'
MODEL_PATH = 'guberai'

model_path = MODEL_PATH if path.exists(MODEL_PATH) else PRETRAINED_MODEL_PATH

tokenizer = GPT2Tokenizer.from_pretrained(PRETRAINED_MODEL_PATH)
model = GPT2LMHeadModel.from_pretrained(model_path).to(DEVICE)  # type: ignore

In [ ]:
EPOCHS = 30
LR = 1e-5
BATCH_SIZE = 32
WARMUP_STEPS = 10
GRADIENT_ACCUMULATION_STEPS = 16

train_dataset = TextDataset(
    tokenizer=tokenizer, file_path=FILENAME, block_size=BATCH_SIZE
)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    load_best_model_at_end=False,
    num_train_epochs=EPOCHS,
    output_dir=MODEL_PATH,
    overwrite_output_dir=True,
    per_device_eval_batch_size=BATCH_SIZE,
    per_device_train_batch_size=BATCH_SIZE,
    save_steps=1,
    save_total_limit=1,
    warmup_steps=WARMUP_STEPS
)
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers=(torch.optim.AdamW(model.parameters(), lr=LR), None)  # type: ignore
)

trainer.train()
trainer.save_model()

## Тестирование модели

In [ ]:
MAX_LENGTH = 60
NUM_BEAMS = 2
TEMPERATURE = 1.6
TOP_P = 0.9

INPUTS = [
    'За радости любовных ощущений\nОднажды острой болью заплатив,',
    'Люблю людей и по наивности\nОткрыто с ними говорю.',
    'Глупо думать про лень негативно\nИ надменно о ней отзываться:',
    'Всему ища вину вовне,\nЯ злился так, что лез из кожи,',
    'Давно уже две жизни я живу,\nОдной — внутри себя, другой — наружно;',
    'Не зря я пью вино на склоне дня,\nЗаслужена его глухая власть;',
    'Эта мысль — украденный цветок,\nПросто рифма ей не повредит:',
    'Однажды летом в январе\nСлона увидел я в ведре,',
    'Бывают лампы в сотни ватт,\nНо свет их резок и увечен,',
    'Когда мы раздражаемся и злы,\nОбижены, по сути, мы на то,'
]

model.eval()

for element in INPUTS:
    input_ids = tokenizer.encode(element, return_tensors='pt').to(DEVICE)

    with torch.no_grad():
        out = model.generate(
            input_ids,
            do_sample=True,
            num_beams=NUM_BEAMS,
            temperature=TEMPERATURE,
            top_p=TOP_P,
            max_length=MAX_LENGTH
        )
    generated_text, *_ = (tokenizer.decode(token) for token in out)
    poem = build_poem(generated_text)

    print(poem)
    print('-' * 20)